# AG3 - Actividad Guiada 3

### Nombre: Mikel Escobar de Carlos

https://github.com/Mikelesc/03MIAR_Algoritmos-de-Optimizacion

https://colab.research.google.com/drive/1JJ65UR1-o3jxQZha6r0GHRIavQ-vccyP?usp=sharing


Carga de los datos del problema

In [1]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp

#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion : 
  # http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/en/stable/pages/usage.html
  # https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
  # https://pypi.org/project/tsplib95/

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ; 
urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file) 

#Coordendas 51-city problem (Christofides/Eilon)
#file = "eil51.tsp" ; urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/eil51.tsp", file)

#Coordenadas - 48 capitals of the US (Padberg/Rinaldi)
#file = "att48.tsp" ; urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/att48.tsp", file)




('swiss42.tsp', <http.client.HTTPMessage at 0x1bc0bcb65c8>)

In [2]:
#Modulos extras, no esenciales
import numpy as np
import matplotlib.pyplot as plt
import imageio                    #Para construir las imagenes con gif
# from google.colab import files    #Para descargar ficheros generados con google colab
from tempfile import mkstemp      #Para genera carpetas y ficheros temporales
import random                     #Para generar valores aleatorios

In [3]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())



In [4]:
#Probamos algunas funciones del objeto problem

#Distancia entre nodos
problem.get_weight(0, 1)

#Todas las funciones
#Documentación: https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
                
#dir(problem)

15

### Funcionas basicas


In [5]:
 
#Funcionas basicas
###############################################################################      
     
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos): 
    solucion = [Nodos[0]]
    for n in Nodos[1:]:
        solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
    return solucion 

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
    return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
    distancia_total = 0
    for i in range(len(solucion)-1):
        distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
    return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

### Busqueda Aleatoria

In [6]:
###############################################################################
# BUSQUEDA ALEATORIA
###############################################################################

def busqueda_aleatoria(problem, N):
  Nodos = list(problem.get_nodes())
  
  mejor_solucion = []
  #mejor_distancia = 10e100                         #Inicializamos con un valor alto
  mejor_distancia = float('inf')                    #Inicializamos con un valor alto
  
  for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
    solucion = crear_solucion(Nodos)                #Genera una solucion aleatoria
    distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo(distancia total)
    
    if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
      mejor_solucion = solucion
      mejor_distancia = distancia
      
      
  print("Mejor solución:" , mejor_solucion) 
  print("Distancia     :" , mejor_distancia) 
  return mejor_solucion  

    
#Busqueda aleatoria con 5000 iteraciones    
solucion = busqueda_aleatoria(problem, 5000)    
 

Mejor solución: [0, 13, 17, 12, 11, 8, 34, 33, 18, 35, 27, 22, 28, 30, 5, 37, 36, 15, 16, 31, 29, 19, 9, 40, 24, 21, 39, 4, 23, 6, 3, 26, 32, 41, 25, 2, 1, 10, 14, 7, 38, 20]
Distancia     : 3807


#Busqueda Local  2-opt

In [7]:
###############################################################################
# BUSQUEDA LOCAL
###############################################################################

def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Se puede modificar para aplicar otros generadores distintos que 2-opt
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):
      
      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]  

      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)         

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion


#solucion = [1, 47, 13, 41, 40, 19, 42, 44, 37, 5, 22, 28, 3, 2, 29, 21, 50, 34, 30, 9, 16, 11, 38, 49, 10, 39, 33, 45, 15, 24, 43, 26, 31, 36, 35, 20, 8, 7, 23, 48, 27, 12, 17, 4, 18, 25, 14, 6, 51, 46, 32]
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))
 

nueva_solucion = genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))

 

Distancia Solucion Incial: 3807
Distancia Mejor Solucion Local: 3432


In [16]:
#Busqueda Local:
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem):
  mejor_solucion = []
  
  #Generar una solucion inicial de referencia(aleatoria)
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)

  iteracion=0             #Un contador para saber las iteraciones que hacemos
  while(1):
    iteracion +=1         #Incrementamos el contador
    #print('#',iteracion)

    #Obtenemos la mejor vecina ...
    vecina = genera_vecina(solucion_referencia)

    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)
 
    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      #mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia 
      mejor_solucion = vecina                   #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina

    else:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion) 
      print("Distancia     :" , mejor_distancia) 
      return mejor_solucion
      
    solucion_referencia = vecina
 
 
sol = busqueda_local(problem )  


En la iteracion  32 , la mejor solución encontrada es: [0, 32, 34, 28, 10, 41, 23, 29, 30, 38, 22, 25, 11, 12, 20, 33, 35, 36, 31, 27, 2, 8, 9, 21, 40, 24, 39, 18, 13, 19, 5, 26, 1, 7, 17, 37, 15, 16, 14, 6, 4, 3]
Distancia     : 2010


#Simulated Annealing

In [9]:
###############################################################################
# SIMULATED ANNEALING
###############################################################################

#Generador de 1 solucion vecina 2-opt 100% aleatoria (intercambiar 2 nodos)
#Mejorable eligiendo otra forma de elegir una vecina.
def genera_vecina_aleatorio(solucion):

  #Se eligen dos nodos aleatoriamente
  i,j = sorted(random.sample( range(1,len(solucion)) , 2))
  
  #Devuelve una nueva solución pero intercambiando los dos nodos elegidos al azar
  return solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
  
 
genera_vecina_aleatorio(solucion) 


[0,
 13,
 17,
 12,
 11,
 8,
 34,
 21,
 18,
 35,
 27,
 22,
 28,
 30,
 5,
 37,
 36,
 15,
 16,
 31,
 29,
 19,
 9,
 40,
 24,
 33,
 39,
 4,
 23,
 6,
 3,
 26,
 32,
 41,
 25,
 2,
 1,
 10,
 14,
 7,
 38,
 20]

In [10]:

#Funcion de probabilidad para aceptar peores soluciones
def probabilidad(T,d):
  if random.random() <  math.exp( -1*d / T)  :
    return True
  else:
    return False

#Funcion de descenso de temperatura
def bajar_temperatura(T):
  return T*0.99

In [11]:
def recocido_simulado(problem, TEMPERATURA ):
  #problem = datos del problema
  #T = Temperatura
  
  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  
  N=0
  while TEMPERATURA > .0001:
    N+=1
    #Genera una solución vecina
    vecina =genera_vecina_aleatorio(solucion_referencia)
    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
      
    #Si es la mejor solución de todas se guarda(siempre!!!)
    if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina
    
    #Si la nueva vecina es mejor se cambia  
    #Si es peor se cambia según una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina)
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) ) :
      #solucion_referencia = copy.deepcopy(vecina)
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina

    #Bajamos la temperatura
    TEMPERATURA = bajar_temperatura(TEMPERATURA)
 
  print("La mejor solución encontrada es " , end="")
  print(mejor_solucion)
  print("con una distancia total de " , end="")
  print(mejor_distancia)
  return mejor_solucion

solu  = recocido_simulado(problem, 30000000)

La mejor solución encontrada es [0, 1, 11, 25, 10, 22, 38, 34, 33, 20, 30, 23, 41, 8, 29, 32, 3, 27, 28, 39, 24, 40, 21, 9, 2, 26, 5, 13, 18, 12, 4, 6, 15, 16, 14, 19, 37, 36, 35, 31, 17, 7]
con una distancia total de 1973


### Tareas opcionales para mejorar nota:
 - Búsqueda local con Entornos variables.
 ¿Se puede mejorar con otros operadores de vecindad variables?


Voy a probar no a cambiar parejas contiguas sino a una distancia aleatoria.

In [20]:
# def swap_random(seq): # Intercambia de manera aleatoria dos elementos de la lista
#     idx = range(len(seq))
#     i1, i2 = random.sample(idx, 2)
#     seq[i1], seq[i2] = seq[i2], seq[i1]
#     return seq

def genera_vecina2(solucion, n_iter):
    
    mejor_solucion = solucion
    mejor_distancia = 10e100
    for i in range(n_iter):

        vecina = genera_vecina_aleatorio(solucion)

        #Se evalua la nueva solución
        distancia_vecina = distancia_total(vecina, problem)         

        # guardarla si mejora las anteriores
        if distancia_vecina < mejor_distancia:
            print("Hemos encontrado solucion con distancia: ", distancia_vecina)
            mejor_distancia = distancia_vecina
            mejor_solucion = vecina

    return mejor_solucion, mejor_distancia


#Busqueda aleatoria con 5000 iteraciones    
# solucion = busqueda_aleatoria(problem, 5000)
print("Distancia Solucion Aleatoria:" , distancia_total(solucion, problem))
 

sol, dist = genera_vecina2(solucion, 200000)
print("\nDistancia Mejor Solucion Local permutaciones no contiguas:", dist)


Distancia Solucion Aleatoria: 4736
Hemos encontrado solucion con distancia:  4773
Hemos encontrado solucion con distancia:  4531
Hemos encontrado solucion con distancia:  4432
Hemos encontrado solucion con distancia:  4400
Hemos encontrado solucion con distancia:  4364
Hemos encontrado solucion con distancia:  4361
Hemos encontrado solucion con distancia:  4309

Distancia Mejor Solucion Local permutaciones no contiguas: 4309


No conseguimos el mejor resultado pero bueno. 

Voy a combinar las busqueda local por parejas contiguas con esta. La idea es que cuando la busqueda local permutando parejas no pueda mejorar nuestra solucion, intentar buscar un cambio aleatorio que si que lo consiga, para salir de ese minimo local o intentarlo e intensificar de nuevo intercambiando parejas.

In [47]:
def genera_vecina_aleatorio2(solucion, problem):
    init_d = distancia_total(solucion, problem)
    #Se eligen dos nodos aleatoriamente
    k = 0
    while k < 5000:
        k += 1
        i,j = sorted(random.sample( range(1,len(solucion)) , 2))
        s = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
        new_d = distancia_total(s, problem)
        if new_d < init_d:
            print("\nDistancia nueva aleatoria! :" , new_d) 
            return s     #Devuelve una nueva solución pero intercambiando los dos nodos elegidos al azar, solo si mejora la existente
    
    return solucion

def busqueda_local2(solucion, problem):
    #Generar una solucion inicial de referencia(aleatoria)
    mejor_sol = solucion
    mejor_distancia = distancia_total(mejor_sol, problem)

    iteracion=0             #Un contador para saber las iteraciones que hacemos
    while(1):
        iteracion +=1         #Incrementamos el contador
        #print('#',iteracion)

        #Obtenemos la mejor vecina ...
        vecina = genera_vecina(mejor_sol)

        #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
        distancia_vecina = distancia_total(vecina, problem)

        #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
        if distancia_vecina < mejor_distancia:
            #mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia 
            mejor_sol = vecina                   #Guarda la mejor solución encontrada
            mejor_distancia = distancia_vecina

        else:
            if iteracion == 60:
                print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_sol)
                print("Distancia     :" , mejor_distancia) 
                return mejor_sol
            else:
                mejor_sol = genera_vecina_aleatorio2(mejor_sol, problem) # Si no hemos llegado a la iteracion 60 hacemos una busqueda aleatoria a ver si encontramos algo mejor

            

# solucion = [1, 47, 13, 41, 40, 19, 42, 44, 37, 5, 22, 28, 3, 2, 29, 21, 50, 34, 30, 9, 16, 11, 38, 49, 10, 39, 33, 45, 15, 24, 43, 26, 31, 36, 35, 20, 8, 7, 23, 48, 27, 12, 17, 4, 18, 25, 14, 6, 51, 46, 32]
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))
 

sol2 = busqueda_local2(solucion, problem)
print("Distancia Mejor Solucion Local:", distancia_total(sol2,problem))

Distancia Solucion Incial: 4736
initd 1408
initd 1408
initd 1408
initd 1408
initd 1408
initd 1408
initd 1408
initd 1408
initd 1408
initd 1408
initd 1408
initd 1408
initd 1408
initd 1408
initd 1408
initd 1408
initd 1408
initd 1408
initd 1408
Distancia Mejor Solucion Local: 1408


No consigo mejorar la busqueda local normal.

 ### Recocido simulado
 ### ¿Se puede mejorar con otra elección no tan aleatoria( función genera_vecina_aleatorio() ) ?

Si, durante mis estudios de Física Computacional usabamos un criterio más complejo que dependía de la temperatura para aceptar las nuevas configuraciones. En concreto hicimos una simulación de Monte Carlo del modelo de Ising, la cual optimizabamos con simmulated annealing.